In [1]:
import json
import pandas as pd
from shapely.geometry import Point, shape
from multiprocessing.dummy import Pool as ThreadPool

In [2]:
#Load the BMTC Routes information
bmtc_routes = pd.read_csv('routes.csv')
bmtc_routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Data columns (total 13 columns):
Unnamed: 0                    1982 non-null int64
route_no                      1982 non-null object
distance                      1982 non-null object
origin                        1982 non-null object
destination                   1982 non-null object
map_link                      1982 non-null object
busstops_link                 1982 non-null object
time                          1982 non-null object
map_json_content              1974 non-null object
departure_from_origin         1982 non-null object
arrival_at_destination        1982 non-null object
departure_from_destination    1982 non-null object
arrival_at_origin             1982 non-null object
dtypes: int64(1), object(12)
memory usage: 201.4+ KB


In [3]:
#Drop columns with no information
bmtc_routes.drop(['Unnamed: 0', 'map_link', 'busstops_link'], axis='columns', inplace=True)
bmtc_routes.head()

,route_no,distance,origin,destination,time,map_json_content,departure_from_origin,arrival_at_destination,departure_from_destination,arrival_at_origin
0,1,19.6 KM,Jayanagara 9th Block,Yeshwanathapura new railway station,01:25 Min.,"[{""busstop"": ""Jayanagara 9th Block,JAYANAGARA ...","07:35, 08:15, 09:35, 10:40, 11:20, 12:40,...","08:50, 09:30, 10:50, 11:55, 12:35, 13:55,...","08:55, 09:35, 11:20, 12:00, 12:40, 14:00, 14:4...","10:10, 10:50, 12:35, 13:15, 13:55, 15:15, 15:5..."
1,AS-1,6.5 KM,Banashankari Bus Station,Avalahalli BDA Layout,00:25 Min.,"[{""busstop"": ""Banashankari Bus Station"", ""latl...","07:40, 08:05, 08:25, 08:40, 09:15, 09:25,...","08:05, 08:35, 08:50, 09:05, 09:45, 09:50,...","08:10, 08:40, 08:55, 09:10, 09:50, 10:20, 10:3...","08:35, 09:10, 09:20, 09:35, 10:15, 10:45, 11:0..."
2,BVK-1,8 KM,Kempegowda Bus Station,Kempegowda Bus Station,00:55 Min.,"[{""busstop"": ""Kempegowda Bus Station"", ""latlon...","10:40, 11:40, 12:40, 13:40, 15:05, 16:05,...","11:35, 12:35, 13:35, 14:35, 16:00, 17:00,...","10:40, 11:40, 12:40, 13:40, 15:05, 16:05, 17:0...","11:35, 12:35, 13:35, 14:35, 16:00, 17:00, 18:0..."
3,FDR-1,19 KM,Vasanthapura,Vasanthapura,01:20 Min.,"[{""busstop"": ""Vasanthapura,VASANTHAPURA,IN FRO...","06:15, 06:50, 07:40, 08:15, 09:30, 10:05,...","07:35, 08:10, 09:00, 09:35, 10:50, 11:25,...","06:15, 06:50, 07:40, 08:15, 09:30, 10:05, 10:5...","07:35, 08:10, 09:00, 09:35, 10:50, 11:25, 12:1..."
4,KBS-1I,28.4 KM,Kempegowda Bus Station,Kadugodi,01:35 Min.,"[{""busstop"": ""Kempegowda Bus Station"", ""latlon...","04:30, 04:45, 05:00, 05:15, 05:30, 05:45,...","06:05, 06:20, 06:35, 06:50, 07:05, 07:20,...","04:30, 04:45, 05:00, 05:15, 05:30, 05:45, 06:0...","06:05, 06:20, 06:35, 06:50, 07:05, 07:20, 07:3..."


In [16]:
bmtc_routes['map_json_content'].iloc[0]

'[{"busstop": "Jayanagara 9th Block,JAYANAGARA 9TH BLOCK,BESIDE HOTEL SRI SAI", "latlons": ["12.9197565816171", "77.5923588994416"]}, {"busstop": "Jayanagara T Block,JAYANAGARA 4TH T BLOCK,BESIDE HDFC BANK", "latlons": ["12.9226893214618", "77.5933845451659"]}, {"busstop": "Jayanagara 18th Main,JAYANAGARA 4TH T BLOCK,OPPOSITE OF LAKSHMI AYURVEDIC CENTER", "latlons": ["12.9231859777075", "77.5887716805949"]}, {"busstop": "Church,JAYANAGARA 4TH T BLOCK,BESIDE POORNIMA CONVENTION", "latlons": ["12.9279596029171", "77.5876041867249"]}, {"busstop": "Jayanagara 4th Block,TTMC", "latlons": ["12.9284826437007", "77.58420959115"]}, {"busstop": "Jayanagara 3rd Block,JAYANAGARA 3RD BLOCK,OPPOSITE OF UTL TECHNOLOGIS", "latlons": ["12.9317783285", "77.5839063618"]}, {"busstop": "Madavan Park,JAYANAGARA 3RD BLOCK,OPPOSITE OF ORIENTAL INSUARENCE", "latlons": ["12.9361068271", "77.5839236286"]}, {"busstop": "Southend Circle,BASAVANAGUDI,BESIDE GANDHARVA FAMILY BAR", "latlons": ["12.9366643075", "77.57

In [4]:
bus_stops_set = set()

In [5]:
# Mapping betwen route number and bus stop coordinates
# Look at one such example for Route number '1'
bus_stop_coordinates = bmtc_routes['map_json_content']
bus_stops_json = {}
for idx, stop_coordinates in enumerate(bus_stop_coordinates):
    bus_stops_json[str(bmtc_routes['route_no'].iloc[idx]).strip()] = stop_coordinates


In [6]:
stop_sum = 0

In [7]:
bus_routes = bmtc_routes['route_no']
for route in bus_routes:
    try:
        route_map_json = json.loads(bus_stops_json[str(route).strip()])
        stop_sum += len(route_map_json)
        for entry in route_map_json:
            bus_stops_set.add(tuple(entry['latlons']))
    except Exception as e:
        print ('Cannot load bus number {}'.format(route))

Cannot load bus number   CS-7
Cannot load bus number   PK-18
Cannot load bus number   PK-45D
Cannot load bus number   JPV-114A
Cannot load bus number   144H
Cannot load bus number   210WA
Cannot load bus number   265F
Cannot load bus number   401F


In [8]:
stop_sum

57575

In [9]:
len(bus_stops_set)

6145

In [10]:
bus_stops_set

{('13.0514681251193', '77.7848906880579'),
 ('12.9604612', '77.55693463'),
 ('12.911304', '77.480297'),
 ('12.92398165', '77.60022159'),
 ('13.1724789731928', '77.5645217693503'),
 ('12.9611695558', '77.5688594207'),
 ('12.7414340246', '77.4883395061'),
 ('13.2429971404638', '77.4479642244305'),
 ('13.19998', '77.84959'),
 ('12.9890949944636', '77.5376666541049'),
 ('12.9975471164749', '77.546009060451'),
 ('13.00462', '77.59976'),
 ('12.7488213312', '77.4244843237'),
 ('13.065520196031', '77.4544564526615'),
 ('12.9571125488595', '77.7685066656666'),
 ('12.9986799974', '77.5563046709'),
 ('12.9823997082408', '77.6927989865896'),
 ('12.98972419', '77.59934214'),
 ('12.97739', '77.60818'),
 ('12.9945985765912', '77.6269619416357'),
 ('13.121532714', '77.610931294'),
 ('12.931768', '77.425001'),
 ('13.0572077681152', '77.4601837299526'),
 ('13.0170412268', '77.5058460329'),
 ('12.9380825240388', '77.5337524901504'),
 ('13.0231551030165', '77.5841606504848'),
 ('12.9703019468099', '77.566

In [11]:
#Get wardnames from the BBMP json file
with open('bbmp-wards.json') as fp:
    wards_json = json.load(fp)
ward_features = wards_json['features']
#Initialize number of stops in each ward to zero
for idx, var in enumerate(ward_features):
    ward_features[idx]['properties']['number_of_bus_stops'] = 0

In [12]:
def number_of_bus_stops_in_ward(bus_stops_set):
    '''
    wards_features: The ward level geojson file
    bus_stop_coordinates_geojson: Mapping between route number and the latlong
    
    return: Number of bus stops in a ward as part of the wards_geojson file
    '''
    pool = ThreadPool(6)
    # for route_number, details in bus_stop_coordinates_json.items():
    pool.map(populate_for_route, bus_stops_set)
    # populate_for_route(details)
    return ward_features

In [14]:
def populate_for_route(details):
    #location = json.loads(details)
    for lat_long_tuple in details:
        bus_stop = Point(float(lat_long_tuple[1]), float(lat_long_tuple[0]))
        #print ('Bus stop coordinates is {}'.format(bus_stop))
        for idx, ward_feature in enumerate(ward_features):
            ward_coordinates = shape(ward_features[idx]['geometry'])
            #print ('Ward coordinates is {}'.format(ward_coordinates))
            if bus_stop.within(ward_coordinates):
                known_bus_stops = ward_features[idx]['properties'].get('bus_stop_coordinates', None)
                if known_bus_stops is None:
                    known_bus_stops = {'bus_stops': [lat_long]}
                    ward_features[idx]['properties']['number_of_bus_stops'] += 1
                elif lat_long in known_bus_stops.values():
                    continue
                else:
                    known_bus_stops['bus_stops'].append(lat_long)
                    ward_features[idx]['properties']['number_of_bus_stops'] += 1
                ward_features[idx]['properties']['bus_stop_coordinates'] = known_bus_stops
            else:
                continue

In [ ]:
updated_ward_details = number_of_bus_stops_in_ward(bus_stops_set)

In [ ]:
checksum = 0

In [177]:
for idx, var in enumerate(ward_features):
    checksum += ward_features[idx]['properties']['number_of_bus_stops']
    #print (idx, ward_features[idx]['properties']['Ward_Name'], ward_features[idx]['properties']['number_of_bus_stops'])

In [178]:
checksum

0